In [29]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [30]:
env = GridworldEnv()

In [31]:
# Based on the Iterative Policy Evaluation presented in Section 4.1 of Sutton and Barto.
# It based on "in place" updation of the value function
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)        
    while True:             
        delta = 0
        # In a given state s,
        for s in range(env.nS):            
            v = V[s]
            v_temp = 0
            # If agent performs action a
            for a in range(env.nA):
                # Then the environment can blow it in different successor states
                # leading to different reward
                for ns_prob, next_state, reward, done in env.P[s][a]:                    
                    # Update the state-value "in place"
                    v_temp += (policy[s, a] * ns_prob * (reward + discount_factor * V[next_state]))
            
            V[s] = v_temp                        
            delta = max(delta, np.abs(v - V[s]))        
            
        if delta < theta:            
            break
    return np.array(V)

In [32]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

In [33]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)